# Unsupervised Machine Scoring of Free Response Answers—Validated Against Law School Final Exams

This notebook implements the methods described in [Unsupervised Machine Scoring of Free Response Answers—Validated Against Law School Final Exams](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4040303), to be presented at the [Computational Legal Studies Conference](https://cclaw.smu.edu.sg/events/computational-legal-studies-2022), March 2022, hosted by the Center for Computational Law at Singapore Management University.

Paper Summary

> This paper presents a novel method for unsupervised machine scoring of short answer and essay question responses, relying solely on a sufficiently large set of responses to a common prompt, absent the need for pre-labeled sample answers—given said prompt is of a particular character. That is, for questions where “good” answers look similar, “wrong” answers are likely to be “wrong” in different ways. Consequently, when a collection of text embeddings for responses to a common prompt are placed in an appropriate feature space, the centroid of their placements can stand in for a model answer, providing a lodestar against which to measure individual responses. This paper examines the efficacy of this method and discusses potential applications.
>
>Current methods for the automated scoring of short answer and essay questions are poorly suited to spontaneous and idiosyncratic assessments. That is, the time saved in grading must be balanced against the time required for the training of a model. This includes tasks such as the creation of pre-labeled sample answers. This limits the utility of machine grading for single classes working with novel assessments. The method described here eliminates the need for the preparation of pre-labeled sample answers. It is the author’s hope that such a method may be leveraged to reduce the time needed to grade free response questions, promoting the increased adoption of formative assessment esp. in contexts like law school instruction which traditionally have relied almost exclusively on summative assessments.
>
>Ranking by the algorithm is found to be statistically significant when compared to a pseudo-random shuffle. To determine how similar a list’s order was to that produced by a human grader, the lowest number of neighbor swaps needed to transform the ordering of these lists into that of the human ordering was calculated. For a dataset including more than one thousand student answers to a set of thirteen free response questions, drawn from six Suffolk University Law School final exams, taught by five instructors, the p-value for a paired t-test of the two populations’ swaps, with the pseudo-random group acting as the untreated group and the machine-grader acting as the treatment, came to 0.000000977, allowing us to reject the null hypothesis that the machine's ordering is equivalent to a random shuffle. Additionally, the Cohen’s d for the number of swaps between the pseudo-random ordering and machine ordering was found to be large (i.e., 0.986).

This notebook was used to obtain the results referenced above. Student exam answers and their associated grade data were acquired and this research conducted after approval was granted by Suffolk University’s Office of Research and Sponsored Programs (ORSP) which oversees all human subject research at Suffolk University.

This notebook implements the primary steps of the method described in the paper, namely:  

1. Produce an embedding for each answer that captures as much of the relevant information as possible.
2. Find the centroid for all of the embeddings in your population of answers and impute the location of a  “correct” model answer.
3. Measure the distance between each answer’s embedding and the “correct” answer (e.g., the answers’ centroid or medoid).
4. Convert the answers’ distances from the model answer into z-scores for the population of answers.
5. Translate these z-scores into some known grading scale.
6. Order the answers according to this scale.
7. Compare these orderings to random orderings by seeing how many times you have to change their rankings to obtain the same ordering as one would get if they were ordered by their human-assigned grades.

_Note: The methods described here are the subject of USPTO Patent Application Serial No. 17/246,563._

## Contents

_These, and other internal links (e.g., "back to contents"), may not work if you are viewing the preview of this notebook on GitHub.com. Links to other notebooks, external sites, and exam questions should, however, be okay._

This notebook has three sections:

- [Data: Student Exam Answers](#Data:-Student-Exam-Answers)
- [Code:Code: Libraries and Novel Functions](#Code:-Libraries-and-Novel-Functions)
- [Results](#Results)


## Data: Student Exam Answers

More than one thousand exam answers were obtained as PDF files containing answers to thirteen free response questions, drawn from six Suffolk University Law School final exams, taught by five instructors. Each PDF corresponded to a single student exam. These PDFs were parsed to extract their answers, their contents converted into XML of the following format.

```
<EXAM>
    <STUDENT id='ID'>00000000</STUDENT>
    <QUESTION id='Q1'>
        <![CDATA[
            text of written answer to question one
        ]]>
    </QUESTION>
    <QUESTION id='Q2'>
        <![CDATA[
            text of written answer to question two
        ]]>
    </QUESTION>
    <QUESTION id='Q3'>
        <![CDATA[
            text of written answer to question three
        ]]>
    </QUESTION>
</EXAM>
```

These translated files were reviewed by hand and reformatted as needed to correct any formatting errors. Each XML file was then read into a csv file for its associated exam as a single row with their columns corresponding to each of its elements (e.g., ID, Q1, Q2). Additionally, a column stating the number of words contained in each question was appended to the csv file (e.g., size_Q1, size_Q2). E.g., 

|ID|Q1|Q2|size_Q1|size_Q2|
|--|--|--|-------|-------|
|00001|text of 1's ans to q1|text of 1's ans to q2|6|6|
|00002|text of 2's ans to q1|text of 2's ans to q2|6|6|

Instructors also provided scores for each exam question. These were placed in a csv for each exam with the scores on each question associated to the exam ID. E.g., 

|ID|Q1|Q2|
|--|--|--|
|00001|96|93|
|00002|83|89|

A PDF did not extist for every exam as some exams were hand-written. Also, for the _Property Instructor A_ exam, the IDs of two PDFs did not macth any of the IDs provided in the score sheet. To address such issues, you will see that we merge the two data tables above using the default behavior of Panda's merge method (i.e., we kept only the intersection of ID, those found in both files). Consequently, we did not score the exams for which no PDF existed or those for which we could not find a score on the intructor-provided list of scores.

To see the code used to assist in this processing see the following notebook: [Prep Exams](Prep%20Exams.ipynb).

In keeping with the wishes of those instructors who provided exams, only three of the exams are available here for review (i.e., [property_instructor_A](https://colarusso.github.io/free-response-scoring/exam_questions/property_instructor_A.docx), [property_instructor_B](https://colarusso.github.io/free-response-scoring/exam_questions/property_instructor_B.docx), and [crim_instructor_E](https://colarusso.github.io/free-response-scoring/exam_questions/crim_instructor_E.docx)). Another two are on file with the author and may be shared upon request and the assent of their authors. In keeping with the author's wishes, the remaining exam will not be shared. Subject to constraints imposed by the Family Educational Rights and Privacy Act (FERPA), at least three of the answer sets may be shared upon request. One of these is linked to an exam requiering instructor assent to be shared. I am in the process of reviewing and cleaning these data for PII. You can track my progress [here](https://twitter.com/Colarusso/status/1494781654237425666?s=20&t=g0Zt5f3wDnVys6oH3nVSGQ).

Please note that the exam questiones shared at the links above are docx files and have been redacted to exclude the instructor's name and text that does not include the scored question prompts (e.g., multiple choice questions). 

To facilitate retrieval of the exam answers which were stored in various folders, a list of dictionaries is defined. Each dictionary defines the folder name where csv files can be found as well as the names of its relevant columns for later consideration (i.e., their ID and those questions to be scored). To avoid sharing of this data publicly, as seen below, these files are not included in this repository and were located outside of this repository's folder (i.e., `../data/`). 

In [1]:
exams = [
            {"folder":"../data/property_instructor_A","columns":["ID","SHORT_ANS","Q1","Q2"]},
            {"folder":"../data/property_instructor_B","columns":["ID","Q1","Q2"]},
            {"folder":"../data/environ_instructor_B","columns":["ID","Q2"]},
            {"folder":"../data/PR_instructor_C","columns":["ID","Q1","Q2"]},
            {"folder":"../data/contracts_instructor_D","columns":["ID","Q1","Q2","Q3"]},
            {"folder":"../data/crim_instructor_E","columns":["ID","Q1","Q2"]}
          ]

[back to contents](#Contents)

## Code: Libraries and Novel Functions

### Python Libraries et al.

First we'll load the following libraries. These are needed to perform our scoring. 

In [2]:
import csv
import pandas as pd
import numpy as np
from numpy import var
from math import sqrt
import spacy
from scipy import stats
from sklearn.preprocessing import normalize

# https://spacy.io/models/en#en_core_web_lg
nlp1 = spacy.load('en_core_web_lg') 
# https://github.com/explosion/spacy-transformers/tree/88814f5f4be7f0d4c784d8500c558d9ba06b9a56
nlp2 = spacy.load("en_trf_distilbertbaseuncased_lg") 
nlp3 = spacy.load("en_trf_robertabase_lg") 

In [3]:
def score_exams(exams,model='nlp1',normv=0,score=0,goal="centroid",runs=1):

    # list of swaps needed to move from machine-scored ordering to that of the graded exams
    M = []  
    # list of swaps needed to move from machine-scored ordering to that of the graded exams
    R = []

    # for every exam in the list of exams  
    for exam in exams:

        print("\n=========================================================")
        print(exam["folder"])
        print("=========================================================\n")

        # load this exam's answer texts
        texts = pd.read_csv("%s/texts_redacted.csv"%exam["folder"], encoding="utf-8")
        # load this exam's grades
        actual = pd.read_csv("%s/actual_redacted.csv"%exam["folder"], encoding="utf-8")

        # print the relevant columns for this exam (i.e., ID and questions to score)
        print(exam["columns"])

        # score the questions listed above
        output_df = grade(texts[texts["ID"].isin(actual["ID"])][exam["columns"]],model,normv,score,goal)
        
        # below we want the average word counts for questions
        # so we need a df with only the questions actually used here
        # i.e., the inner merge. output_df doesn't have the texts columns
        # so we'll do a second merge here real quick to facilitate that count
        texts_df = actual.merge(texts, on="ID")

        # only include grades for those questions defined above
        actual = actual[[x for x in actual.columns if x in exam["columns"]]]

        # merge the machine-scored dataframe with that of the actual grades
        df_exam = actual.merge(output_df, on="ID")
        
        # display a preview of the two scores (first few rows of the merged dataframe)
        # I'm excluding the ID here to avoid sharing IDs in the cell's output
        display(df_exam[[x for x in list(df_exam.columns.values) if (x not in ["ID"])]].head())

        print("Number of entries:",len(df_exam))

        # for each of the questions that were scored
        for qn in [x for x in exam["columns"] if (x not in ["ID"])]:
            mean_words = int(round(texts_df["size_%s"%qn].mean()))
            print("\n%s\nMean Words:"%qn,mean_words)

            # create a dataframe with the grade and machine scores for this question
            df = df_exam[["ID", "%s_x"%qn, "%s_y"%qn]]

            # ideally the granularity of the machine and human scoring should be the same
            # to avoid false precision. It may be worth imposing this in some manner 
            # simmilar to this comment. 
            # df["%s_y"%qn] = round(df["%s_x"%qn].max()*(df["%s_y"%qn]/100))

            # sort answers by the machine scores
            df = df.sort_values(by='%s_y'%qn, ascending=True)

            # create an list of these scores then count the swaps needed to bring them into 
            # agreement with the grades. Append this to our list of swaps needed to move from 
            # machine-scored ordering to that of the graded exams
            arr_ = df[df.columns[1]].values.copy()
            n = len(arr_)
            M_o = countSwaps(arr_, n)
            M.append(M_o)

            # we're going to do the same as the above but for pseudo-random shuffles.
            # we'll shuffle and count swaps for the number of times defined in `runs`
            # and average the results 
            tmp = []
            for x in range(runs):
                # note the use of `sample.` This is where we get our pseudo-random ordering
                arr_ = df[df.columns[1]].sample(frac=1).values.copy()
                n = len(arr_)
                R_o = countSwaps(arr_, n)
                tmp.append(R_o)

            R_o = np.array(tmp).mean()
            R.append(R_o)

            print("Algo score, swaps needed:",M_o)
            print("Swaps needed for first 10 out of %s pseudo-random runs:"%runs)
            print(tmp[:10])
            print("Pseudo-random, avergae swaps needed:",R_o)

    print("\n=========================================================")
    print("Swaps (Machine):")
    print(M)
    print("N:",len(M),"\tMean: ",np.mean(M),"\tVar: ",np.var(M))
    print("Avergae swaps (Pseudo-random):")
    print(R)
    print("N:",len(R),"\tMean: ",np.mean(R),"\tVar: ",np.var(R),"\n")

    # Print out a summary table of the percent difference between pseudo-random and machine swaps
    pdiff = []
    i = 0
    for x in R:
        if x != 0 and M[i] != 0:
            pdiff_ = 100*(x-M[i])/((x+M[i])/2)
            pdiff.append(pdiff_)
            print( i+1,")",x,">",M[i],"=>",pdiff_ )
        else:
            print("error")
        i = i + 1

    print("\nAvergae percent difference between swaps needed for pseudo-random and machine sorting:")
    print(np.array(pdiff).mean(),"\n")
        
    # run t test for the two populations of swaps
    sample_n = len(M)
    print(stats.ttest_rel(M[:sample_n], R[:sample_n]))
    # return the effect size
    print("Cohen's d: ",cohend(R[:sample_n], M[:sample_n]))

In [4]:
def grade(df,model,normv,score=0,goal="centroid"):
       
    df = df.copy()    
    
    # exclude the ID column to get only the answer texts
    questions = [x for x in list(df.columns.values) if (x not in ["ID"])]
    # vectorize the text of each answer
    for x in questions:
        df["%s_vec"%x] = df[x].apply(vectorize, args=(model,normv)) 
    
    # for each question
    for x in questions:
        
        # based on the `goal` parameter grade, against either the centroid or medoid
        # that is, set either as the model answer aginst which to grade others
        if goal=="centroid":
            # centroid 
            best_ans = df["%s_vec"%x].mean()
        else:
            # medoid 
            best_ans = df["%s_vec"%x].iloc[best_answer(df,x)]

        # measure each answer's distance from the model answer above
        df["%s_dist"%x] = df["%s_vec"%x].apply(distance, args=(best_ans,))
        
        # replace the text of the answer with its z-score
        df[x] =  df["%s_vec"%x].apply(z_score, args=(df,x,best_ans))
        
        # translate the z-score into a standard grade using
        # the translation defined by `score`
        if score==0:
            df[x] =  df[x].apply(score_0)
        elif score==1:
            df[x] =  df[x].apply(score_1)
            
    # create output df without the vectors or distance measures
    output_df = df[[ x for x in list(df.columns.values) if ("_vec" not in x) and ("_dist" not in x) ]].copy()
            
    return output_df

In [5]:
def vectorize(row,model='nlp1',normv=0): 

    # use a method for vectorization based on the `model` parameter
    # note: some methods can work across the entire text of a qustion
    # others requier vectorization at the sentence level
    if model=='nlp1':
        nlp = nlp1
        mode = 0
    elif model=='nlp2':
        nlp = nlp2
        mode = 1
    elif model=='nlp3':
        nlp = nlp3
        mode = 1
        
    try:
        if mode == 1:
            # vectorize at the sentence level
            doc = nlp(row)
            tmp_df = pd.DataFrame({'sent' : []})
            i=0 
            for sent in doc.sents:
                sent_vec = nlp(sent.string.strip()).vector
                # if normv=1 normalize each sentence vector
                if normv==1:
                    sent_vec = norm(sent_vec)
                if i==0:
                    tmp_df = pd.DataFrame([[sent_vec]],columns=["sent"])
                else:
                    tmp_df = tmp_df.append(pd.DataFrame([[sent_vec]],columns=["sent"]), ignore_index=True)
                i+=1
            # average sentence vectors 
            output = tmp_df["sent"].mean()
        else:
            # vectorize all input text together
            output = nlp(row).vector

        # if normv=1 normalize the answer's vector
        if normv==1:
            output = norm(output)  
    
        return output
    except:
        return np.NaN
    
def norm(row):
    # normalize vector
    matrix = row.reshape(1,-1).astype(np.float64)
    return normalize(matrix, axis=1, norm='l1')[0]

In [6]:
def best_answer(df,column):
    # Return the medoid (like the centroid, but confined to a member of the group) 
    # i.e., the best answer in the group of answers.
    tmp_df = df[[ "%s_vec"%column ]].copy()
    centroid = tmp_df["%s_vec"%column].mean()
    tmp_df["%s_dist"%column] = tmp_df["%s_vec"%column].apply(distance, args=(centroid,))
    tmp_df = tmp_df[tmp_df["%s_dist"%column]==tmp_df["%s_dist"%column].min()]
    return tmp_df.index[0]

def distance(row,centroid):
    # calculate the Euclidean distance between centroid and row
    # See https://stackoverflow.com/a/1401828
    return np.linalg.norm(centroid-row)

In [7]:
def z_score(row,df,x,best):
    # calculate the z-score for a given distance
    return -1*(np.linalg.norm(best-row)-df["%s_dist"%x].mean())/df["%s_dist"%x].std()

In [8]:
# Define grade bin boarders based on z-score
# --------------------------------------
# There's no standard mapping between z-scores and letter grades
# However, it's common for schools to set their own curve which 
# is in the same spirit. Therefor I have created a mapping where
# the average answer recieves a B. Others might choose differently

grade_bins = [
              2,    # A+
              1.5,  # A
              1,    # A-
              0.5,  # B+
              0,    # B
              -0.5, # B-
              -1,   # C+
              -1.5, # C
              -2,   # C-
              -2.5, # D+
              -3,   # D
              -3.5  # D-
             ]

# Assign numeric grades based on z-score
# -----------------------------------------
# This was the original z-score to percentage grade translation.
# I took the above and translated it into numberic grades
# with the scale bottoming out at 59 and topping out at 97
def score_0(row):
    if row >= grade_bins[0]:
        grade = 97
    elif row >= grade_bins[1]:
        grade = 93+((row-grade_bins[1])/(grade_bins[0]-grade_bins[1]))*4
    elif row >= grade_bins[2]:
        grade = 90+((row-grade_bins[2])/(grade_bins[1]-grade_bins[2]))*3
    elif row >= grade_bins[3]:
        grade = 87+((row-grade_bins[3])/(grade_bins[2]-grade_bins[3]))*3
    elif row >= grade_bins[4]:
        grade = 83+((row-grade_bins[4])/(grade_bins[3]-grade_bins[4]))*4
    elif row >= grade_bins[5]:
        grade = 80+((row-grade_bins[5])/(grade_bins[4]-grade_bins[5]))*3
    elif row >= grade_bins[6]:
        grade = 77+((row-grade_bins[6])/(grade_bins[5]-grade_bins[6]))*3
    elif row >= grade_bins[7]:
        grade = 73+((row-grade_bins[7])/(grade_bins[6]-grade_bins[7]))*4
    elif row >= grade_bins[8]:
        grade = 70+((row-grade_bins[8])/(grade_bins[7]-grade_bins[8]))*3
    elif row >= grade_bins[9]:
        grade = 67+((row-grade_bins[9])/(grade_bins[8]-grade_bins[9]))*3
    elif row >= grade_bins[10]:
        grade = 63+((row-grade_bins[10])/(grade_bins[9]-grade_bins[10]))*4
    elif row >= grade_bins[11]:
        grade = 60+(row-grade_bins[11])*(3/(grade_bins[10]-grade_bins[11]))
    else:
        grade = 59
        
    # The final output is rounded to avoid false percision. A better 
    # approach is to bin all of the scores. This is done in score_1 below
    return round(grade)

# Assign numeric grades based on z-score
# -----------------------------------------
# Upon subsequent reflection, I realized that binned scores were preferable
# to a continuous score with dropoffs at 59 and 97. The above translation,
# however, was used on the first run of this code and so is included here
# for compleatness.  
def score_1(row):
    if row >= grade_bins[0]:
        grade = 97
    elif row >= grade_bins[1]:
        grade = 93
    elif row >= grade_bins[2]:
        grade = 90
    elif row >= grade_bins[3]:
        grade = 87
    elif row >= grade_bins[4]:
        grade = 83
    elif row >= grade_bins[5]:
        grade = 80
    elif row >= grade_bins[6]:
        grade = 77
    elif row >= grade_bins[7]:
        grade = 73
    elif row >= grade_bins[8]:
        grade = 70
    elif row >= grade_bins[9]:
        grade = 67
    elif row >= grade_bins[10]:
        grade = 63
    elif row >= grade_bins[11]:
        grade = 60
    else:
        grade = 59
    return grade

In [9]:
# The following functions are used to count the number of adjacent swaps needed
# to change one ordering into another. It was written by Shivam Gupta
# See https://www.geeksforgeeks.org/number-swaps-sort-adjacent-swapping-allowed/
# -----------------------------------------------------
# python 3 program to count number of swaps required 
# to sort an array when only swapping of adjacent 
# elements is allowed. 
# include <bits/stdc++.h> 

#This function merges two sorted arrays and returns inversion count in the arrays.*/ 
def merge(arr, temp, left, mid, right): 
    inv_count = 0

    i = left #i is index for left subarray*/ 
    j = mid #i is index for right subarray*/ 
    k = left #i is index for resultant merged subarray*/ 
    while ((i <= mid - 1) and (j <= right)): 
        if (arr[i] <= arr[j]): 
            temp[k] = arr[i] 
            k += 1
            i += 1
        else: 
            temp[k] = arr[j] 
            k += 1
            j += 1

            #this is tricky -- see above explanation/ 
            # diagram for merge()*/ 
            inv_count = inv_count + (mid - i) 

    #Copy the remaining elements of left subarray 
    # (if there are any) to temp*/ 
    while (i <= mid - 1): 
        temp[k] = arr[i] 
        k += 1
        i += 1

    #Copy the remaining elements of right subarray 
    # (if there are any) to temp*/ 
    while (j <= right): 
        temp[k] = arr[j] 
        k += 1
        j += 1

    # Copy back the merged elements to original array*/ 
    for i in range(left,right+1,1): 
        arr[i] = temp[i] 

    return inv_count 

#An auxiliary recursive function that sorts the input 
# array and returns the number of inversions in the 
# array. */ 
def _mergeSort(arr, temp, left, right): 
    inv_count = 0
    if (right > left): 
        # Divide the array into two parts and call 
        #_mergeSortAndCountInv() 
        # for each of the parts */ 
        mid = int((right + left)/2) 

        #Inversion count will be sum of inversions in 
        # left-part, right-part and number of inversions 
        # in merging */ 
        inv_count = _mergeSort(arr, temp, left, mid) 
        inv_count += _mergeSort(arr, temp, mid+1, right) 

        # Merge the two parts*/ 
        inv_count += merge(arr, temp, left, mid+1, right) 

    return inv_count 

#This function sorts the input array and returns the 
#number of inversions in the array */ 
def countSwaps(arr, n): 
    temp = [0 for i in range(n)] 
    return _mergeSort(arr, temp, 0, n - 1) 

In [10]:
# The following function calculates Cohen d (the effect size)
# based on two populations. It was written by Jason Brownlee
# See https://machinelearningmastery.com/effect-size-measures-in-python/
# function to calculate Cohen's d for independent samples
def cohend(d1, d2):
    # calculate the size of samples
    n1, n2 = len(d1), len(d2)
    # calculate the variance of the samples
    s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1)
    # calculate the pooled standard deviation
    s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    # calculate the means of the samples
    u1, u2 = np.mean(d1), np.mean(d2)
    # calculate the effect size
    return (u1 - u2) / s

[back to contents](#Contents)

# Results

The arrangement of parameters below corresponds to those used when all of the exam data was first run through the above scoring method. To avoid biasing parameter selection, prior work on the method did NOT use the exam data used here. Prior drafts of the associated paper made use of data from this first run, placing it in one of two tables. Note: the below cell has been re-run since this first run. The second table contained output from the cell following the one below, where the pseudo-random shuffle was repeated 100,000 times, and the number of swaps needed were averaged. The current version of the paper has opted to include only the latter table as much of the majority of data was duplicated across tables, and the 100,000 average was deemed a better representation of the method's performance.

In [11]:
score_exams(exams,model='nlp1',normv=0,score=0,goal="centroid",runs=1)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,86,83,87
1,10,30,32,88,86,87
2,12,39,24,90,90,85
3,15,38,16,89,87,86
4,13,43,19,91,89,87


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1002
Swaps needed for first 10 out of 1 pseudo-random runs:
[1708]
Pseudo-random, avergae swaps needed: 1708.0

Q1
Mean Words: 2048
Algo score, swaps needed: 1021
Swaps needed for first 10 out of 1 pseudo-random runs:
[1460]
Pseudo-random, avergae swaps needed: 1460.0

Q2
Mean Words: 947
Algo score, swaps needed: 828
Swaps needed for first 10 out of 1 pseudo-random runs:
[1571]
Pseudo-random, avergae swaps needed: 1571.0

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,88,86
1,20.0,18.0,80,84
2,24.0,25.0,89,80
3,18.0,22.0,83,91
4,17.0,17.0,78,79


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1290
Swaps needed for first 10 out of 1 pseudo-random runs:
[1841]
Pseudo-random, avergae swaps needed: 1841.0

Q2
Mean Words: 1546
Algo score, swaps needed: 1326
Swaps needed for first 10 out of 1 pseudo-random runs:
[1485]
Pseudo-random, avergae swaps needed: 1485.0

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 911
Swaps needed for first 10 out of 1 pseudo-random runs:
[1306]
Pseudo-random, avergae swaps needed: 1306.0

Q2
Mean Words: 570
Algo score, swaps needed: 795
Swaps needed for first 10 out of 1 pseudo-random runs:
[1189]
Pseudo-random, avergae swaps needed: 1189.0

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,87,87,81
1,53,27,27,82,80,89
2,38,36,20,81,84,81
3,66,37,37,83,88,87
4,71,34,38,85,88,85


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1178
Swaps needed for first 10 out of 1 pseudo-random runs:
[1558]
Pseudo-random, avergae swaps needed: 1558.0

Q2
Mean Words: 1050
Algo score, swaps needed: 993
Swaps needed for first 10 out of 1 pseudo-random runs:
[1258]
Pseudo-random, avergae swaps needed: 1258.0

Q3
Mean Words: 839
Algo score, swaps needed: 807
Swaps needed for first 10 out of 1 pseudo-random runs:
[1267]
Pseudo-random, avergae swaps needed: 1267.0

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,89,89
1,24.0,25.0,82,85
2,26.5,24.0,63,75
3,23.5,23.0,87,87
4,26.5,21.0,87,90


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1792
Swaps needed for first 10 out of 1 pseudo-random runs:
[2080]
Pseudo-random, avergae swaps needed: 2080.0

Q2
Mean Words: 2137
Algo score, swaps needed: 1637
Swaps needed for first 10 out of 1 pseudo-random runs:
[2123]
Pseudo-random, avergae swaps needed: 2123.0

Swaps (Machine):
[1002, 1021, 828, 1290, 1326, 115, 911, 795, 1178, 993, 807, 1792, 1637]
N: 13 	Mean:  1053.4615384615386 	Var:  162956.55621301773
Avergae swaps (Pseudo-random):
[1708.0, 1460.0, 1571.0, 1841.0, 1485.0, 164.0, 1306.0, 1189.0, 1558.0, 1258.0, 1267.0, 2080.0, 2123.0]
N: 13 	Mean:  1462.3076923076924 	Var:  223687.75147928996 

1 ) 1708.0 > 1002 => 52.103321033210335
2 ) 1460.0 > 1021 => 35.38895606610238
3 ) 1571.0 > 828 => 61.94247603167987
4 ) 1841.0 > 1290 => 35.19642286809326
5 ) 1485.0 > 1326 => 11.312700106723586
6 ) 164.0 > 115 => 35.12544802867384
7 ) 1306.0 > 911 => 35.63373928732521
8 ) 1189.0 > 795 => 39.71774193548387
9 ) 155

As stated above, the following cell is the same as the above except the pseudo-random shuffle was run 100,000 times and the swaps needed averaged. The data presented in Table 1 of the current paper is drawn from this cell. 

In [12]:
score_exams(exams,model='nlp1',normv=0,score=0,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,86,83,87
1,10,30,32,88,86,87
2,12,39,24,90,90,85
3,15,38,16,89,87,86
4,13,43,19,91,89,87


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1002
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1685, 1723, 1331, 1398, 1397, 1501, 1452, 1484, 1385, 1645]
Pseudo-random, avergae swaps needed: 1499.63409

Q1
Mean Words: 2048
Algo score, swaps needed: 1021
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1411, 1523, 1531, 1737, 1441, 1481, 1593, 1483, 1569, 1504]
Pseudo-random, avergae swaps needed: 1566.37027

Q2
Mean Words: 947
Algo score, swaps needed: 828
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1679, 1463, 1495, 1590, 1423, 1511, 1568, 1489, 1748, 1621]
Pseudo-random, avergae swaps needed: 1559.28464

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,88,86
1,20.0,18.0,80,84
2,24.0,25.0,89,80
3,18.0,22.0,83,91
4,17.0,17.0,78,79


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1290
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1951, 1866, 1747, 1763, 1849, 1862, 1828, 1840, 1990, 1680]
Pseudo-random, avergae swaps needed: 1837.79401

Q2
Mean Words: 1546
Algo score, swaps needed: 1326
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1771, 1827, 1769, 1619, 1830, 1856, 1698, 1726, 2052, 2065]
Pseudo-random, avergae swaps needed: 1757.98221

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 911
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1391, 1291, 1510, 1413, 1532, 1435, 1443, 1331, 1251, 1386]
Pseudo-random, avergae swaps needed: 1324.99583

Q2
Mean Words: 570
Algo score, swaps needed: 795
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1211, 1398, 1284, 1142, 1412, 1364, 1206, 1198, 1043, 1267]
Pseudo-random, avergae swaps needed: 1220.1239

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,87,87,81
1,53,27,27,82,80,89
2,38,36,20,81,84,81
3,66,37,37,83,88,87
4,71,34,38,85,88,85


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1178
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1487, 1541, 1274, 1497, 1417, 1421, 1357, 1456, 1465, 1386]
Pseudo-random, avergae swaps needed: 1472.14792

Q2
Mean Words: 1050
Algo score, swaps needed: 993
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1085, 1615, 1486, 1415, 1256, 1515, 1500, 1556, 1472, 1495]
Pseudo-random, avergae swaps needed: 1449.32104

Q3
Mean Words: 839
Algo score, swaps needed: 807
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1290, 1448, 1594, 1619, 1607, 1438, 1463, 1460, 1388, 1388]
Pseudo-random, avergae swaps needed: 1458.47715

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,89,89
1,24.0,25.0,82,85
2,26.5,24.0,63,75
3,23.5,23.0,87,87
4,26.5,21.0,87,90


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1792
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2099, 2053, 2060, 2063, 1911, 2067, 2301, 2042, 2055, 1945]
Pseudo-random, avergae swaps needed: 2045.33843

Q2
Mean Words: 2137
Algo score, swaps needed: 1637
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1716, 1887, 1969, 1792, 2063, 1905, 2076, 2160, 2124, 1911]
Pseudo-random, avergae swaps needed: 1994.05218

Swaps (Machine):
[1002, 1021, 828, 1290, 1326, 115, 911, 795, 1178, 993, 807, 1792, 1637]
N: 13 	Mean:  1053.4615384615386 	Var:  162956.55621301773
Avergae swaps (Pseudo-random):
[1499.63409, 1566.37027, 1559.28464, 1837.79401, 1757.98221, 182.5335, 1324.99583, 1220.1239, 1472.14792, 1449.32104, 1458.47715, 2045.33843, 1994.05218]
N: 13 	Mean:  1489.8503976923078 	Var:  198343.47024575656 

1 ) 1499.63409 > 1002 => 39.78472247314154
2 ) 1566.37027 > 1021 => 42.156337368752396
3 ) 1559.28464 > 828 => 61.26497257570426
4 ) 1837.79

The following cells present a variety of parameter selections, including a mix of vector normalization, grade translations, "best answer" selections, and the number of pseudo-random runs.

In [13]:
score_exams(exams,model='nlp1',normv=1,score=0,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,87,85,86
1,10,30,32,87,83,86
2,12,39,24,89,90,85
3,15,38,16,88,86,86
4,13,43,19,92,89,87


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1009
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1473, 1374, 1544, 1648, 1485, 1487, 1610, 1520, 1478, 1423]
Pseudo-random, avergae swaps needed: 1499.17097

Q1
Mean Words: 2048
Algo score, swaps needed: 1035
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1329, 1669, 1761, 1573, 1714, 1688, 1541, 1566, 1575, 1689]
Pseudo-random, avergae swaps needed: 1566.21067

Q2
Mean Words: 947
Algo score, swaps needed: 937
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1472, 1646, 1675, 1646, 1498, 1631, 1475, 1477, 1569, 1601]
Pseudo-random, avergae swaps needed: 1559.86743

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,85,83
1,20.0,18.0,79,83
2,24.0,25.0,89,79
3,18.0,22.0,83,92
4,17.0,17.0,82,79


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1255
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1899, 1752, 1888, 1875, 1835, 1918, 2030, 1649, 1756, 1749]
Pseudo-random, avergae swaps needed: 1837.5727

Q2
Mean Words: 1546
Algo score, swaps needed: 1215
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1867, 1694, 1774, 2087, 1715, 1521, 1875, 2068, 1704, 1751]
Pseudo-random, avergae swaps needed: 1757.71688

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 921
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1454, 1342, 1409, 1204, 1472, 1347, 1158, 1063, 1332, 1319]
Pseudo-random, avergae swaps needed: 1324.36787

Q2
Mean Words: 570
Algo score, swaps needed: 891
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1150, 1188, 1158, 1220, 1223, 1230, 1276, 1174, 1158, 1090]
Pseudo-random, avergae swaps needed: 1219.29825

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,87,85,80
1,53,27,27,81,79,89
2,38,36,20,81,83,81
3,66,37,37,82,88,87
4,71,34,38,85,91,83


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1189
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1491, 1336, 1601, 1187, 1546, 1561, 1549, 1526, 1290, 1589]
Pseudo-random, avergae swaps needed: 1471.94899

Q2
Mean Words: 1050
Algo score, swaps needed: 1049
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1373, 1565, 1305, 1594, 1425, 1552, 1400, 1518, 1417, 1382]
Pseudo-random, avergae swaps needed: 1449.67106

Q3
Mean Words: 839
Algo score, swaps needed: 814
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1379, 1469, 1316, 1473, 1502, 1287, 1538, 1303, 1250, 1457]
Pseudo-random, avergae swaps needed: 1458.02596

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,90,91
1,24.0,25.0,81,84
2,26.5,24.0,61,74
3,23.5,23.0,85,85
4,26.5,21.0,86,91


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1658
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2010, 1772, 1949, 2155, 1827, 1992, 2055, 1938, 1937, 2021]
Pseudo-random, avergae swaps needed: 2045.16105

Q2
Mean Words: 2137
Algo score, swaps needed: 1580
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1735, 2103, 2293, 1952, 2001, 2126, 1732, 1906, 1988, 2095]
Pseudo-random, avergae swaps needed: 1993.17556

Swaps (Machine):
[1009, 1035, 937, 1255, 1215, 113, 921, 891, 1189, 1049, 814, 1658, 1580]
N: 13 	Mean:  1051.2307692307693 	Var:  133029.10059171595
Avergae swaps (Pseudo-random):
[1499.17097, 1566.21067, 1559.86743, 1837.5727, 1757.71688, 182.48577, 1324.36787, 1219.29825, 1471.94899, 1449.67106, 1458.02596, 2045.16105, 1993.17556]
N: 13 	Mean:  1489.590243076923 	Var:  198301.65356998646 

1 ) 1499.17097 > 1009 => 39.08592961667201
2 ) 1566.21067 > 1035 => 40.8433408432851
3 ) 1559.86743 > 937 => 49.89191036065539
4 ) 1837.572

In [14]:
score_exams(exams,model='nlp1',normv=1,score=1,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,87,83,83
1,10,30,32,83,83,83
2,12,39,24,87,87,83
3,15,38,16,87,83,83
4,13,43,19,90,87,83


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1028
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1415, 1531, 1386, 1256, 1227, 1459, 1541, 1573, 1508, 1425]
Pseudo-random, avergae swaps needed: 1499.26688

Q1
Mean Words: 2048
Algo score, swaps needed: 1065
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1745, 1489, 1646, 1442, 1480, 1522, 1554, 1700, 1608, 1508]
Pseudo-random, avergae swaps needed: 1566.47909

Q2
Mean Words: 947
Algo score, swaps needed: 931
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1427, 1539, 1507, 1376, 1525, 1732, 1492, 1478, 1432, 1528]
Pseudo-random, avergae swaps needed: 1560.36679

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,83,83
1,20.0,18.0,77,80
2,24.0,25.0,87,77
3,18.0,22.0,80,90
4,17.0,17.0,80,77


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1338
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1694, 1916, 1847, 1901, 1685, 1711, 1817, 1930, 2033, 2003]
Pseudo-random, avergae swaps needed: 1837.61571

Q2
Mean Words: 1546
Algo score, swaps needed: 1254
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1609, 1844, 1718, 1663, 1733, 1887, 1668, 1769, 1532, 1679]
Pseudo-random, avergae swaps needed: 1757.0195

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 904
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1271, 1327, 1422, 1423, 1251, 1339, 1406, 1310, 1179, 1377]
Pseudo-random, avergae swaps needed: 1325.39077

Q2
Mean Words: 570
Algo score, swaps needed: 1046
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1288, 1136, 1195, 1416, 1266, 1290, 1169, 1250, 1241, 1242]
Pseudo-random, avergae swaps needed: 1219.72981

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,87,83,80
1,53,27,27,80,77,87
2,38,36,20,80,80,80
3,66,37,37,80,87,83
4,71,34,38,83,90,80


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1174
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1449, 1685, 1372, 1491, 1577, 1425, 1423, 1516, 1446, 1441]
Pseudo-random, avergae swaps needed: 1472.6937

Q2
Mean Words: 1050
Algo score, swaps needed: 1084
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1333, 1406, 1519, 1514, 1419, 1358, 1425, 1289, 1297, 1364]
Pseudo-random, avergae swaps needed: 1449.03059

Q3
Mean Words: 839
Algo score, swaps needed: 784
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1556, 1387, 1428, 1465, 1375, 1608, 1633, 1571, 1462, 1440]
Pseudo-random, avergae swaps needed: 1458.07464

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,90,90
1,24.0,25.0,80,83
2,26.5,24.0,60,73
3,23.5,23.0,83,83
4,26.5,21.0,83,90


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1627
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2354, 2438, 2050, 2115, 2166, 1937, 2136, 2186, 2186, 2252]
Pseudo-random, avergae swaps needed: 2046.16375

Q2
Mean Words: 2137
Algo score, swaps needed: 1622
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1902, 2032, 1782, 1968, 1865, 1881, 1787, 2222, 2112, 2164]
Pseudo-random, avergae swaps needed: 1994.08258

Swaps (Machine):
[1028, 1065, 931, 1338, 1254, 102, 904, 1046, 1174, 1084, 784, 1627, 1622]
N: 13 	Mean:  1073.7692307692307 	Var:  138424.79289940826
Avergae swaps (Pseudo-random):
[1499.26688, 1566.47909, 1560.36679, 1837.61571, 1757.0195, 182.48329, 1325.39077, 1219.72981, 1472.6937, 1449.03059, 1458.07464, 2046.16375, 1994.08258]
N: 13 	Mean:  1489.8766999999998 	Var:  198398.38642936078 

1 ) 1499.26688 > 1028 => 37.29458758229759
2 ) 1566.47909 > 1065 => 38.11385710079878
3 ) 1560.36679 > 931 => 50.523816286400766
4 ) 1837.

In [15]:
score_exams(exams,model='nlp1',normv=1,score=1,goal="medoid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,77,77,83
1,10,30,32,80,77,83
2,12,39,24,87,83,83
3,15,38,16,83,87,97
4,13,43,19,87,87,83


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1250
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1451, 1686, 1447, 1497, 1431, 1456, 1380, 1429, 1618, 1388]
Pseudo-random, avergae swaps needed: 1499.96599

Q1
Mean Words: 2048
Algo score, swaps needed: 1236
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1367, 1850, 1422, 1666, 1530, 1345, 1457, 1518, 1413, 1539]
Pseudo-random, avergae swaps needed: 1566.54585

Q2
Mean Words: 947
Algo score, swaps needed: 1021
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1643, 1527, 1656, 1590, 1381, 1369, 1558, 1630, 1540, 1629]
Pseudo-random, avergae swaps needed: 1560.45838

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,83,83
1,20.0,18.0,80,77
2,24.0,25.0,87,80
3,18.0,22.0,80,90
4,17.0,17.0,80,87


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1292
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1887, 2006, 1358, 1626, 1783, 1761, 1526, 1890, 1790, 2047]
Pseudo-random, avergae swaps needed: 1837.39122

Q2
Mean Words: 1546
Algo score, swaps needed: 1607
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1807, 1817, 2067, 1884, 1735, 1896, 1990, 1898, 1866, 1327]
Pseudo-random, avergae swaps needed: 1758.08117

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 885
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1164, 1247, 1201, 1290, 1273, 1336, 1249, 1230, 1345, 1159]
Pseudo-random, avergae swaps needed: 1324.92706

Q2
Mean Words: 570
Algo score, swaps needed: 1036
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1318, 1209, 1212, 1204, 1520, 1452, 1197, 1457, 1102, 1291]
Pseudo-random, avergae swaps needed: 1219.41376

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,87,80,77
1,53,27,27,77,80,87
2,38,36,20,77,83,80
3,66,37,37,80,87,83
4,71,34,38,83,90,83


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1283
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1366, 1569, 1422, 1615, 1500, 1328, 1451, 1367, 1603, 1511]
Pseudo-random, avergae swaps needed: 1472.70655

Q2
Mean Words: 1050
Algo score, swaps needed: 1158
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1401, 1561, 1475, 1429, 1517, 1450, 1703, 1509, 1291, 1327]
Pseudo-random, avergae swaps needed: 1450.22717

Q3
Mean Words: 839
Algo score, swaps needed: 927
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1408, 1274, 1376, 1425, 1595, 1587, 1489, 1601, 1538, 1341]
Pseudo-random, avergae swaps needed: 1458.7472

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,83,90
1,24.0,25.0,77,80
2,26.5,24.0,63,70
3,23.5,23.0,80,80
4,26.5,21.0,83,90


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1846
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1830, 2074, 1897, 2181, 2049, 2071, 2051, 2166, 2080, 2020]
Pseudo-random, avergae swaps needed: 2045.89153

Q2
Mean Words: 2137
Algo score, swaps needed: 1645
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1885, 1989, 1983, 2157, 2123, 1897, 2286, 1946, 1850, 1720]
Pseudo-random, avergae swaps needed: 1994.01366

Swaps (Machine):
[1250, 1236, 1021, 1292, 1607, 140, 885, 1036, 1283, 1158, 927, 1846, 1645]
N: 13 	Mean:  1178.923076923077 	Var:  165570.68639053253
Avergae swaps (Pseudo-random):
[1499.96599, 1566.54585, 1560.45838, 1837.39122, 1758.08117, 182.56995, 1324.92706, 1219.41376, 1472.70655, 1450.22717, 1458.7472, 2045.89153, 1994.01366]
N: 13 	Mean:  1490.0722684615384 	Var:  198401.0186867045 

1 ) 1499.96599 > 1250 => 18.179569558967522
2 ) 1566.54585 > 1236 => 23.58897000739524
3 ) 1560.45838 > 1021 => 41.794853961581204
4 ) 183

In [16]:
score_exams(exams,model='nlp2',normv=1,score=1,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,87,83,87
1,10,30,32,87,83,87
2,12,39,24,90,80,83
3,15,38,16,90,83,87
4,13,43,19,90,83,83


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1002
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1543, 1464, 1780, 1471, 1473, 1411, 1327, 1684, 1372, 1255]
Pseudo-random, avergae swaps needed: 1499.82283

Q1
Mean Words: 2048
Algo score, swaps needed: 1162
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1590, 1466, 1539, 1739, 1586, 1655, 1561, 1741, 1493, 1571]
Pseudo-random, avergae swaps needed: 1566.71399

Q2
Mean Words: 947
Algo score, swaps needed: 906
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1539, 1629, 1533, 1512, 1475, 1441, 1712, 1499, 1309, 1522]
Pseudo-random, avergae swaps needed: 1560.96032

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,83,87
1,20.0,18.0,70,80
2,24.0,25.0,90,80
3,18.0,22.0,77,87
4,17.0,17.0,77,80


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1293
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1928, 1724, 1471, 2009, 2006, 1937, 1773, 1786, 1830, 2103]
Pseudo-random, avergae swaps needed: 1837.19208

Q2
Mean Words: 1546
Algo score, swaps needed: 1274
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1888, 1647, 1697, 1641, 1538, 1710, 1570, 1707, 1767, 1738]
Pseudo-random, avergae swaps needed: 1756.81406

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 932
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1338, 1265, 1262, 1449, 1388, 1340, 1576, 1406, 1338, 1186]
Pseudo-random, avergae swaps needed: 1323.87346

Q2
Mean Words: 570
Algo score, swaps needed: 839
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1162, 1151, 1320, 1251, 1059, 1239, 1203, 1283, 1225, 1262]
Pseudo-random, avergae swaps needed: 1219.70725

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,77,80,80
1,53,27,27,83,87,87
2,38,36,20,77,80,80
3,66,37,37,83,83,83
4,71,34,38,83,87,87


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1125
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1400, 1519, 1555, 1570, 1332, 1580, 1374, 1357, 1503, 1515]
Pseudo-random, avergae swaps needed: 1472.63522

Q2
Mean Words: 1050
Algo score, swaps needed: 944
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1670, 1534, 1385, 1393, 1349, 1378, 1487, 1505, 1502, 1449]
Pseudo-random, avergae swaps needed: 1449.54875

Q3
Mean Words: 839
Algo score, swaps needed: 843
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1449, 1397, 1445, 1562, 1588, 1454, 1402, 1349, 1490, 1501]
Pseudo-random, avergae swaps needed: 1458.54131

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,93,90
1,24.0,25.0,87,87
2,26.5,24.0,77,80
3,23.5,23.0,90,87
4,26.5,21.0,83,83


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1751
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2337, 2246, 2096, 2229, 2157, 1881, 2015, 1895, 2026, 1989]
Pseudo-random, avergae swaps needed: 2045.6723

Q2
Mean Words: 2137
Algo score, swaps needed: 1716
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1772, 2073, 2226, 2055, 1864, 1996, 2142, 1972, 2038, 2139]
Pseudo-random, avergae swaps needed: 1994.47099

Swaps (Machine):
[1002, 1162, 906, 1293, 1274, 87, 932, 839, 1125, 944, 843, 1751, 1716]
N: 13 	Mean:  1067.2307692307693 	Var:  163186.94674556216
Avergae swaps (Pseudo-random):
[1499.82283, 1566.71399, 1560.96032, 1837.19208, 1756.81406, 182.63844, 1323.87346, 1219.70725, 1472.63522, 1449.54875, 1458.54131, 2045.6723, 1994.47099]
N: 13 	Mean:  1489.8916153846153 	Var:  198368.41855899102 

1 ) 1499.82283 > 1002 => 39.79680927286127
2 ) 1566.71399 > 1162 => 29.663349950428476
3 ) 1560.96032 > 906 => 53.098569497866904
4 ) 1837.19

In [17]:
score_exams(exams,model='nlp2',normv=1,score=1,goal="medoid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,80,77,83
1,10,30,32,80,80,87
2,12,39,24,83,73,80
3,15,38,16,83,77,97
4,13,43,19,87,77,83


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1357
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1489, 1454, 1565, 1656, 1543, 1329, 1676, 1470, 1643, 1652]
Pseudo-random, avergae swaps needed: 1499.68327

Q1
Mean Words: 2048
Algo score, swaps needed: 1463
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1336, 1430, 1443, 1525, 1689, 1699, 1515, 1754, 1738, 1673]
Pseudo-random, avergae swaps needed: 1566.29709

Q2
Mean Words: 947
Algo score, swaps needed: 1022
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1443, 1539, 1771, 1453, 1696, 1617, 1507, 1573, 1391, 1455]
Pseudo-random, avergae swaps needed: 1559.43937

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,83,87
1,20.0,18.0,73,80
2,24.0,25.0,90,80
3,18.0,22.0,77,87
4,17.0,17.0,77,80


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1230
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2016, 1571, 1933, 1754, 1836, 1868, 1991, 1965, 1858, 1980]
Pseudo-random, avergae swaps needed: 1837.10399

Q2
Mean Words: 1546
Algo score, swaps needed: 1202
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1716, 1567, 1693, 1962, 1784, 1713, 1836, 1827, 1885, 1506]
Pseudo-random, avergae swaps needed: 1757.9422

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 1016
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1351, 1362, 1331, 1261, 1124, 1190, 1321, 1460, 1175, 1261]
Pseudo-random, avergae swaps needed: 1324.90515

Q2
Mean Words: 570
Algo score, swaps needed: 1001
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1279, 1294, 1113, 1118, 1305, 1183, 1047, 1208, 1255, 1229]
Pseudo-random, avergae swaps needed: 1219.75558

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,83,83,80
1,53,27,27,80,83,87
2,38,36,20,80,77,77
3,66,37,37,83,80,83
4,71,34,38,83,87,87


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1066
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1586, 1266, 1689, 1521, 1629, 1475, 1651, 1643, 1419, 1470]
Pseudo-random, avergae swaps needed: 1472.35927

Q2
Mean Words: 1050
Algo score, swaps needed: 1017
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1317, 1526, 1450, 1380, 1333, 1423, 1392, 1270, 1456, 1460]
Pseudo-random, avergae swaps needed: 1450.3711

Q3
Mean Words: 839
Algo score, swaps needed: 758
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1350, 1560, 1437, 1480, 1283, 1471, 1331, 1580, 1621, 1447]
Pseudo-random, avergae swaps needed: 1458.61996

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,90,90
1,24.0,25.0,90,87
2,26.5,24.0,80,80
3,23.5,23.0,87,90
4,26.5,21.0,87,83


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1769
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1946, 1992, 1981, 2116, 2096, 1897, 2118, 1941, 1968, 2033]
Pseudo-random, avergae swaps needed: 2046.21854

Q2
Mean Words: 2137
Algo score, swaps needed: 1696
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1899, 2003, 1896, 2245, 2230, 2016, 1941, 2097, 2012, 2143]
Pseudo-random, avergae swaps needed: 1993.72196

Swaps (Machine):
[1357, 1463, 1022, 1230, 1202, 82, 1016, 1001, 1066, 1017, 758, 1769, 1696]
N: 13 	Mean:  1129.1538461538462 	Var:  169317.20710059168
Avergae swaps (Pseudo-random):
[1499.68327, 1566.29709, 1559.43937, 1837.10399, 1757.9422, 182.43607, 1324.90515, 1219.75558, 1472.35927, 1450.3711, 1458.61996, 2046.21854, 1993.72196]
N: 13 	Mean:  1489.9118115384615 	Var:  198384.91964297523 

1 ) 1499.68327 > 1357 => 9.989435755683198
2 ) 1566.29709 > 1463 => 6.8198718667108364
3 ) 1559.43937 > 1022 => 41.63873660918095
4 ) 183

In [18]:
score_exams(exams,model='nlp3',normv=1,score=1,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,83,87,83
1,10,30,32,87,83,87
2,12,39,24,87,67,80
3,15,38,16,87,87,87
4,13,43,19,80,83,83


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1301
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1614, 1507, 1791, 1455, 1608, 1496, 1565, 1452, 1489, 1380]
Pseudo-random, avergae swaps needed: 1499.73287

Q1
Mean Words: 2048
Algo score, swaps needed: 1254
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1593, 1607, 1304, 1782, 1363, 1544, 1683, 1686, 1328, 1588]
Pseudo-random, avergae swaps needed: 1566.15589

Q2
Mean Words: 947
Algo score, swaps needed: 1189
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1646, 1568, 1559, 1353, 1483, 1679, 1537, 1409, 1497, 1555]
Pseudo-random, avergae swaps needed: 1560.55043

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,87,87
1,20.0,18.0,77,83
2,24.0,25.0,90,87
3,18.0,22.0,83,77
4,17.0,17.0,77,87


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1606
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1977, 2112, 1920, 1831, 1952, 1977, 1637, 1812, 1983, 1769]
Pseudo-random, avergae swaps needed: 1837.41684

Q2
Mean Words: 1546
Algo score, swaps needed: 1570
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1693, 1774, 1815, 1912, 1527, 1865, 1556, 1601, 1727, 1714]
Pseudo-random, avergae swaps needed: 1757.80421

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 927
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1374, 1413, 1373, 1372, 1169, 1274, 1432, 1161, 1297, 1202]
Pseudo-random, avergae swaps needed: 1324.62235

Q2
Mean Words: 570
Algo score, swaps needed: 1013
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1167, 1157, 1333, 1198, 1154, 1261, 1465, 1061, 1326, 1203]
Pseudo-random, avergae swaps needed: 1219.99936

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,80,83,83
1,53,27,27,83,87,83
2,38,36,20,87,83,83
3,66,37,37,83,83,83
4,71,34,38,83,87,80


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1402
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1437, 1560, 1359, 1524, 1333, 1217, 1563, 1442, 1602, 1577]
Pseudo-random, avergae swaps needed: 1472.97963

Q2
Mean Words: 1050
Algo score, swaps needed: 1187
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1659, 1257, 1479, 1335, 1428, 1311, 1397, 1725, 1556, 1248]
Pseudo-random, avergae swaps needed: 1449.73737

Q3
Mean Words: 839
Algo score, swaps needed: 1073
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1561, 1268, 1397, 1495, 1615, 1351, 1477, 1424, 1521, 1429]
Pseudo-random, avergae swaps needed: 1458.14294

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,90,90
1,24.0,25.0,87,83
2,26.5,24.0,77,83
3,23.5,23.0,87,87
4,26.5,21.0,87,87


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1668
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1807, 1873, 1947, 2100, 2206, 1863, 2205, 2010, 1980, 2278]
Pseudo-random, avergae swaps needed: 2045.84129

Q2
Mean Words: 2137
Algo score, swaps needed: 1746
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1708, 1991, 2014, 2147, 1877, 2118, 1879, 2034, 2068, 1965]
Pseudo-random, avergae swaps needed: 1994.34474

Swaps (Machine):
[1301, 1254, 1189, 1606, 1570, 114, 927, 1013, 1402, 1187, 1073, 1668, 1746]
N: 13 	Mean:  1234.6153846153845 	Var:  166348.69822485207
Avergae swaps (Pseudo-random):
[1499.73287, 1566.15589, 1560.55043, 1837.41684, 1757.80421, 182.39369, 1324.62235, 1219.99936, 1472.97963, 1449.73737, 1458.14294, 2045.84129, 1994.34474]
N: 13 	Mean:  1489.9785853846154 	Var:  198432.502839122 

1 ) 1499.73287 > 1301 => 14.191490529405616
2 ) 1566.15589 > 1254 => 22.13749183914794
3 ) 1560.55043 > 1189 => 27.026267708790485
4 ) 1

In [19]:
score_exams(exams,model='nlp3',normv=1,score=1,goal="medoid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,77,83,87
1,10,30,32,87,80,87
2,12,39,24,87,63,83
3,15,38,16,83,80,93
4,13,43,19,87,73,87


Number of entries: 81

SHORT_ANS
Mean Words: 436
Algo score, swaps needed: 1301
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1649, 1618, 1324, 1310, 1182, 1575, 1473, 1518, 1786, 1537]
Pseudo-random, avergae swaps needed: 1499.11033

Q1
Mean Words: 2048
Algo score, swaps needed: 1377
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1543, 1614, 1804, 1304, 1577, 1636, 1232, 1498, 1445, 1668]
Pseudo-random, avergae swaps needed: 1566.59383

Q2
Mean Words: 947
Algo score, swaps needed: 1074
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1612, 1490, 1352, 1513, 1551, 1684, 1506, 1445, 1527, 1513]
Pseudo-random, avergae swaps needed: 1560.06246

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,90,87
1,20.0,18.0,80,77
2,24.0,25.0,90,83
3,18.0,22.0,87,73
4,17.0,17.0,77,87


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1558
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2176, 1927, 2078, 2119, 1820, 1541, 1664, 1898, 1842, 1892]
Pseudo-random, avergae swaps needed: 1837.14275

Q2
Mean Words: 1546
Algo score, swaps needed: 1489
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1760, 1572, 1593, 1829, 1648, 1871, 1873, 1971, 1698, 1693]
Pseudo-random, avergae swaps needed: 1757.42827

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 824
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1262, 1435, 1463, 1270, 1167, 1361, 1435, 1321, 1301, 1620]
Pseudo-random, avergae swaps needed: 1324.48531

Q2
Mean Words: 570
Algo score, swaps needed: 920
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1267, 1143, 1184, 1114, 1269, 1216, 1166, 1390, 1091, 1206]
Pseudo-random, avergae swaps needed: 1219.27619

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,80,83,87
1,53,27,27,83,87,83
2,38,36,20,87,83,83
3,66,37,37,83,87,83
4,71,34,38,87,90,80


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1325
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1383, 1395, 1295, 1348, 1481, 1345, 1590, 1661, 1651, 1545]
Pseudo-random, avergae swaps needed: 1472.85546

Q2
Mean Words: 1050
Algo score, swaps needed: 1179
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1480, 1380, 1411, 1467, 1506, 1358, 1422, 1336, 1315, 1567]
Pseudo-random, avergae swaps needed: 1450.15235

Q3
Mean Words: 839
Algo score, swaps needed: 1102
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1322, 1566, 1510, 1691, 1314, 1573, 1307, 1491, 1480, 1425]
Pseudo-random, avergae swaps needed: 1457.93216

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,93,90
1,24.0,25.0,90,83
2,26.5,24.0,73,83
3,23.5,23.0,87,87
4,26.5,21.0,90,87


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1768
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1744, 2206, 2057, 2260, 2048, 2339, 2173, 2059, 2357, 1941]
Pseudo-random, avergae swaps needed: 2045.61382

Q2
Mean Words: 2137
Algo score, swaps needed: 1714
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1928, 2092, 2368, 1964, 2178, 1785, 1723, 2080, 2029, 1919]
Pseudo-random, avergae swaps needed: 1994.87454

Swaps (Machine):
[1301, 1377, 1074, 1558, 1489, 81, 824, 920, 1325, 1179, 1102, 1768, 1714]
N: 13 	Mean:  1208.6153846153845 	Var:  180965.62130177516
Avergae swaps (Pseudo-random):
[1499.11033, 1566.59383, 1560.06246, 1837.14275, 1757.42827, 182.3896, 1324.48531, 1219.27619, 1472.85546, 1450.15235, 1457.93216, 2045.61382, 1994.87454]
N: 13 	Mean:  1489.8397746153848 	Var:  198456.19869055387 

1 ) 1499.11033 > 1301 => 14.150180289503089
2 ) 1566.59383 > 1377 => 12.881792866103407
3 ) 1560.06246 > 1074 => 36.90591756127151
4 ) 18

[back to contents](#Contents)